In [88]:
import pandas as pd
import datetime
import numpy as np
from collections import defaultdict

In [111]:
texts = pd.read_csv('../data/final_result.txt',sep='+',index_col=0)
texts['date'] = pd.to_datetime(texts['date'])
texts['ticker'] = texts['ticker'].astype('str').map(lambda x: x.zfill(6))

In [112]:
texts = texts[texts.columns[2:]]

In [38]:
texts.loc[:,'weighted_sentiment'] = texts['sentiment']*texts['confidence']

In [39]:
texts = texts.groupby(['ticker','date'])['sentiment','weighted_sentiment'].agg(['sum','count','mean']).reset_index()

In [40]:
texts.columns = ['ticker','date','sentiment_sum','count','sentiment_mean','weighted_sentiment_sum','count2','weighted_sentiment_mean']

In [41]:
texts = texts.drop('count2',1)

In [42]:
texts.head(2)

,ticker,date,sentiment_sum,count,sentiment_mean,weighted_sentiment_sum,weighted_sentiment_mean
0,000036,2018-01-01,2,1,2.0,0.829606,0.829606
1,000036,2018-01-02,2,2,1.0,0.138345,0.069172


In [175]:
uqer = pd.read_csv('../stocks/uqer_price.csv',index_col=0)

In [176]:
uqer.head()

,ticker,secShortName,tradeDate,turnoverVol,turnoverValue
0,36,华联控股,2018-01-02,19918690,1.849531e+08
1,36,华联控股,2018-01-03,16962250,1.567579e+08
2,36,华联控股,2018-01-04,12059040,1.106352e+08
3,36,华联控股,2018-01-05,86175030,8.532058e+08
4,36,华联控股,2018-01-08,86146190,8.773429e+08


In [44]:
uqer['ticker'] = uqer['ticker'].astype('str').map(lambda x: x.zfill(6))
uqer['mean_price'] = uqer['turnoverValue']/uqer['turnoverVol']
uqer = uqer[['ticker','tradeDate','mean_price']]
uqer = uqer.dropna()
uqer['mean_price_change'] = uqer.groupby('ticker')['mean_price'].pct_change()
uqer = uqer.dropna()
uqer.loc[:,'tradeDate'] = pd.to_datetime(uqer['tradeDate']).map(lambda x: x-datetime.timedelta(days=1))

In [231]:
uqer_other = pd.read_csv('../stocks/uqer_other.csv',index_col=0)

In [232]:
uqer_other['ticker'] = uqer_other['ticker'] = uqer_other['ticker'].astype('str').map(lambda x: x.zfill(6))
uqer_other['tradeDate'] = pd.to_datetime(uqer_other['tradeDate'])

In [235]:
uqer_other.head()

,secID,ticker,secShortName,exchangeCD,tradeDate,preClosePrice,actPreClosePrice,openPrice,highestPrice,lowestPrice,...,turnoverRate,accumAdjFactor,negMarketValue,marketValue,chgPct,PE,PE1,PB,isOpen,vwap
0,000036.XSHE,000036,华联控股,XSHE,2017-12-29,9.03,9.03,9.06,9.18,8.92,...,0.0117,0.923664,10264741008,10319048808,0.0011,5.2821,7.1595,2.5063,1,9.061
1,000036.XSHE,000036,华联控股,XSHE,2018-01-02,9.04,9.04,9.00,9.41,9.00,...,0.0175,0.923664,10571320662,10627250487,0.0299,5.4399,7.3733,2.5812,1,9.285
2,000036.XSHE,000036,华联控股,XSHE,2018-01-03,9.31,9.31,9.36,9.36,9.19,...,0.0149,0.923664,10480482246,10535931471,-0.0086,5.3932,7.3100,2.5590,1,9.242
3,000036.XSHE,000036,华联控股,XSHE,2018-01-04,9.23,9.23,9.24,9.26,9.12,...,0.0106,0.923664,10446417840,10501686840,-0.0033,5.3756,7.2862,2.5507,1,9.174
4,000036.XSHE,000036,华联控股,XSHE,2018-01-05,9.20,9.20,9.20,10.12,9.20,...,0.0759,0.923664,11275318386,11334972861,0.0793,5.8022,7.8644,2.7531,1,9.901


In [47]:
uqer_other.columns

Index(['secID', 'ticker', 'secShortName', 'exchangeCD', 'tradeDate',
       'preClosePrice', 'actPreClosePrice', 'openPrice', 'highestPrice',
       'lowestPrice', 'closePrice', 'turnoverVol', 'turnoverValue',
       'dealAmount', 'turnoverRate', 'accumAdjFactor', 'negMarketValue',
       'marketValue', 'chgPct', 'PE', 'PE1', 'PB', 'isOpen', 'vwap'],
      dtype='object')

In [48]:
uqer_other = uqer_other[['ticker','tradeDate','preClosePrice','actPreClosePrice','openPrice','negMarketValue','marketValue','PE','PE1','PB']]

In [49]:
for i in list(set(uqer_other['ticker'].tolist())):
    uqer_other.loc[uqer_other[uqer_other['ticker']==i].index,'PE'] = [0]+uqer_other[uqer_other['ticker']==i]['PE'].tolist()[:-1]

In [50]:
final_output = pd.merge(texts,uqer,how='inner',left_on=['ticker','date'],right_on=['ticker','tradeDate'])

In [51]:
final_output = pd.merge(final_output,uqer_other,how='inner',left_on=['ticker','date'],right_on=['ticker','tradeDate'])

In [52]:
final_output.head(2)

,ticker,date,sentiment_sum,count,sentiment_mean,weighted_sentiment_sum,weighted_sentiment_mean,tradeDate_x,mean_price,mean_price_change,tradeDate_y,preClosePrice,actPreClosePrice,openPrice,negMarketValue,marketValue,PE,PE1,PB
0,000036,2018-01-02,2,2,1.0,0.138345,0.069172,2018-01-02,9.241576,-0.004720,2018-01-02,9.04,9.04,9.0,10571320662,10627250487,5.2821,7.3733,2.5812
1,000036,2018-01-08,0,1,0.0,0.000000,0.000000,2018-01-08,10.367031,0.017937,2018-01-08,9.93,9.93,10.0,11502414426,11563270401,5.8022,8.0228,2.8086


In [53]:
final_output.shape

(4565, 19)

In [20]:
final_output.to_csv('../data/train.csv')

In [57]:
final_output = '\n'.join(','.join(str(x) for x in y) for y in final_output.values)